In [1]:
from transformers import FSMTForConditionalGeneration, FSMTTokenizer

In [2]:
mname = "facebook/wmt19-de-en"
tokenizer = FSMTTokenizer.from_pretrained(mname)
model = FSMTForConditionalGeneration.from_pretrained(mname)

In [3]:
input = "Ich habe mich in eine Essiggurke verwandelt. Ich bin Essiggurke Rick!"

In [4]:
input_ids = tokenizer.encode(input, return_tensors="pt")
outputs = model.generate(input_ids)
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(decoded) 

I have turned into a pickle. I am pickle Rick!
